In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib3
urllib3.disable_warnings()

In [58]:
url = "https://ligaf.es/partido/villareal-cf-femenino/fc-barcelona-femenino/2023309307/eventos"

def scrapper(df, url):
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.content, "html.parser")
    txt = soup.find_all('div', {'class' : 'team local pr mb10'})
    df_local = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])
    df_visitor = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])

    data = []

    local = url.split("/")[4]
    visitor = url.split("/")[5]
    for i in range(len(txt)):
        plchldr = txt[i].get_text('\t').split()
        while "" in plchldr:
            plchldr.replace(" ", "")
        data.append(plchldr)
    for i in range(len(data)):
        ind = i
        possible_events = ["Gol", "puerta", "Asistencia", "Entra", "Sale", "Amarilla", "Roja", "Lesionado"]
        print(data[i])

        while True:
            try:
                minute = int(data[ind][0].replace("'", ""))
                break
            except:
                ind -= 1
                pass
        player = None
        for j in range(len(data[i])):
                for event in possible_events:
                    if data[i][j] == event:
                        if event == "puerta":
                            event_type = "Gol en propia puerta"
                            player =  data[i][j-4]
                        elif event == "Amarilla":
                            event_type = "Tarjeta amarilla"
                            player = data[i][j-2]
                        elif event == "Roja":
                            event_type = "Tarjeta roja"
                            player = data[i][j-2]
                        elif event == "Lesionado":
                            event_type = "Lesionada"
                        else:
                            event_type = event
                            player =  data[i][j-1]
        df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)

    txt = soup.find_all('div', {'class' : 'team visitor pr mb10'})
    data = []

    for i in range(len(txt)):
        plchldr = txt[i].get_text('\t').split()
        while "" in plchldr:
            plchldr.replace(" ", "")
        data.append(plchldr)

    for i in range(len(data)):
        ind = i
        possible_events = ["Gol", "puerta", "Asistencia", "Entra", "Sale", "Amarilla", "Roja", "Lesionado", "Tiro"]
        while True:
            try:
                minute = int(data[ind][0].replace("'", ""))
                break
            except:
                ind -= 1
                pass
        for j in range(len(data[i])):
                for event in possible_events:
                    if data[i][j] == event:
                        if event == "puerta":
                            event_type = "Gol en propia puerta"
                            player =  data[i][j-4]
                        elif event == "Amarilla":
                            event_type = "Tarjeta amarilla"
                            player = data[i][j-2]
                        elif event == "Roja":
                            event_type = "Tarjeta roja"
                            player = data[i][j-2]
                        elif event == "Lesionado":
                            event_type = "Lesionada"
                            player = data[i][j-1]
                        elif event == "Tiro":
                            event_type = "Tiro al palo"
                            player = data[i][j-1]
                        else:
                            event_type = event
                            player =  data[i][j-1]
        df_visitor = df_visitor.append({"Min": minute, "Event": event_type, "Player": player, "Team": visitor, "Against": local, "Home/Away": "Away"}, ignore_index=True)

    return pd.concat([df, df_local, df_visitor])

In [53]:
page = requests.get("https://ligaf.es/equipo/fc-barcelona-femenino/8852/resultados/2023", verify=False)
soup = BeautifulSoup(page.content, "html.parser")
links = soup.find_all('a', {"class":"match-box ta-c"})
df = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])

for link in links:
        url = "https://ligaf.es" + str(link).split('"')[3] + "/eventos"
        #print(url)
        df = scrapper(df, url)

df

C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": play

["47'", 'R.', 'Kundananji', 'Tiro', 'al', 'palo']
['Lauren', 'Tiro', 'al', 'palo']


C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": play

["41'", 'Torrejón', 'Tiro', 'al', 'palo']
["29'", 'I.', 'Paredes', 'Tiro', 'al', 'palo']
["26'", 'Patri', 'Guijarro', 'Tiro', 'al', 'palo']


C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": play

...

["91'", 'C.', 'Pina', 'Tiro', 'al', 'palo']


C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": play

['F.', 'Rolfö', 'Tiro', 'al', 'palo']


C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3057385194.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": play

,Min,Event,Player,Team,Against,Home/Away
0,72,Sale,Hansen,fc-barcelona-femenino,udg_tenerife_,Home
1,72,Entra,Rábano,fc-barcelona-femenino,udg_tenerife_,Home
2,72,Sale,Walsh,fc-barcelona-femenino,udg_tenerife_,Home
3,72,Entra,Engen,fc-barcelona-femenino,udg_tenerife_,Home
4,67,Gol,Hansen,fc-barcelona-femenino,udg_tenerife_,Home
...,...,...,...,...,...,...
7,62,Sale,Fidalgo,real-betis-balompie-femenino,fc-barcelona-femenino,Away
8,62,Entra,Laurina,real-betis-balompie-femenino,fc-barcelona-femenino,Away
9,62,Sale,Salazar,real-betis-balompie-femenino,fc-barcelona-femenino,Away
10,62,Entra,Peña,real-betis-balompie-femenino,fc-barcelona-femenino,Away


In [44]:
df = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])

scrapper(df, url)

C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3739448744.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3739448744.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\3739448744.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": play

,Min,Event,Player,Team,Against,Home/Away
0,79,Sale,García,villareal-cf-femenino,fc-barcelona-femenino,Home
1,79,Entra,Sánchez,villareal-cf-femenino,fc-barcelona-femenino,Home
2,79,Sale,Ainoa,villareal-cf-femenino,fc-barcelona-femenino,Home
3,79,Entra,Bicho,villareal-cf-femenino,fc-barcelona-femenino,Home
4,79,Sale,Sheila,villareal-cf-femenino,fc-barcelona-femenino,Home
5,79,Entra,Estefa,villareal-cf-femenino,fc-barcelona-femenino,Home
6,56,Gol,García,villareal-cf-femenino,fc-barcelona-femenino,Home
7,56,Asistencia,Ainoa,villareal-cf-femenino,fc-barcelona-femenino,Home
8,55,Sale,Cienfuegos,villareal-cf-femenino,fc-barcelona-femenino,Home
9,55,Entra,Llompart,villareal-cf-femenino,fc-barcelona-femenino,Home


In [59]:
df = pd.DataFrame(columns=["Min", "Event", "Player", "Team", "Against", "Home/Away"])
scrapper(df, "https://ligaf.es/partido/fc-barcelona-femenino/levante-las-planas-femenino/2023309436/eventos")

["74'", 'Mariona', 'Gol']
["67'", 'Aitana', 'Entra', 'en', 'el', 'partido']
['Torrejón', 'Sale', 'del', 'partido']
['C.', 'Pina', 'Sale', 'del', 'partido']
['L.', 'Bronze', 'Entra', 'en', 'el', 'partido']
["63'", 'A.', 'Oshoala', 'Gol']
["58'", 'A.', 'Crnogorcevic', 'Sale', 'del', 'partido']
['Emma', 'Entra', 'en', 'el', 'partido']
["56'", 'A.', 'Oshoala', 'Gol']
["48'", 'Vicky', 'López', 'Gol']
['A.', 'Crnogorcevic', 'Asistencia']
["46'", 'Patri', 'Guijarro', 'Sale', 'del', 'partido']
['M.', 'Pérez', 'Entra', 'en', 'el', 'partido']
['Mariona', 'Entra', 'en', 'el', 'partido']
['F.', 'Rolfö', 'Sale', 'del', 'partido']
["41'", 'A.', 'Crnogorcevic', 'Gol']
["33'", 'Vicky', 'López', 'Tiro', 'al', 'palo']
["25'", 'A.', 'Oshoala', 'Gol']
['A.', 'Crnogorcevic', 'Asistencia']
["8'", 'A.', 'Crnogorcevic', 'Gol']
['F.', 'Rolfö', 'Asistencia']


C:\Users\minih\AppData\Local\Temp\ipykernel_7736\114614043.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\114614043.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player, "Team": local, "Against": visitor, "Home/Away": "Home"}, ignore_index=True)
C:\Users\minih\AppData\Local\Temp\ipykernel_7736\114614043.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_local = df_local.append({"Min": minute, "Event": event_type, "Player": player,

,Min,Event,Player,Team,Against,Home/Away
0,74,Gol,Mariona,fc-barcelona-femenino,levante-las-planas-femenino,Home
1,67,Entra,Aitana,fc-barcelona-femenino,levante-las-planas-femenino,Home
2,67,Sale,Torrejón,fc-barcelona-femenino,levante-las-planas-femenino,Home
3,67,Sale,Pina,fc-barcelona-femenino,levante-las-planas-femenino,Home
4,67,Entra,Bronze,fc-barcelona-femenino,levante-las-planas-femenino,Home
5,63,Gol,Oshoala,fc-barcelona-femenino,levante-las-planas-femenino,Home
6,58,Sale,Crnogorcevic,fc-barcelona-femenino,levante-las-planas-femenino,Home
7,58,Entra,Emma,fc-barcelona-femenino,levante-las-planas-femenino,Home
8,56,Gol,Oshoala,fc-barcelona-femenino,levante-las-planas-femenino,Home
9,48,Gol,López,fc-barcelona-femenino,levante-las-planas-femenino,Home
